# İhtisas Komisyonları - Data Collection

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

import pandas as pd
import time
import os

In [2]:
driver = webdriver.Chrome()
driver.implicitly_wait(30)

## İhtisas Komisyonları

- source_page: https://www.tbmm.gov.tr/ihtisas-komisyonlari/Liste

In [3]:
def get_komisyonlar(source_page):

    # Open source page
    driver.get(source_page)
    
    komisyon_tags = driver.find_element(By.CLASS_NAME, "col-md-12").find_element(By.TAG_NAME, "ul").find_elements(By.TAG_NAME, "li")
    
    komisyonlar = []
    urls = []
    
    for komisyon in komisyon_tags:
        komisyonlar.append(komisyon.text)
        urls.append(komisyon.find_element(By.TAG_NAME, "a").get_attribute("href"))
        
    df = pd.concat([pd.Series(komisyonlar, name="komisyon_adi"), pd.Series(urls, name="komisyon_url")], axis=1)
    
    return df

In [4]:
source_page = "https://www.tbmm.gov.tr/ihtisas-komisyonlari/Liste"

In [5]:
komisyonlar_df = get_komisyonlar(source_page)

In [6]:
def get_uye_url(komisyon_url):
    
    driver.get(komisyon_url)
    try:
        uye_url = driver.find_element(By.CLASS_NAME, "menu").find_element(By.TAG_NAME, "ul").find_element(By.LINK_TEXT, "Komisyon Üyeleri").get_attribute("href")
    except:
        uye_url = driver.find_element(By.CLASS_NAME, "blog_right_sidebar").find_element(By.TAG_NAME, "aside").find_element(By.LINK_TEXT, "Komisyon Üyeleri").get_attribute("href")
    return uye_url

In [7]:
komisyonlar_df["uye_url"] = komisyonlar_df["komisyon_url"].apply(get_uye_url)
komisyonlar_df["komisyon_id"] = list(range(1, len(komisyonlar_df)+1))

In [8]:
if "ihtisas_komisyonlari" not in os.listdir():
    os.mkdir("ihtisas_komisyonlari")

komisyonlar_df.to_csv("ihtisas_komisyonlari/ihtisas_komisyonlari.csv", index=False)

In [9]:
# Create urls folder if not exists
if "urls" not in os.listdir():
    os.mkdir("urls")

# Write urls to txt file
with open("urls/ihtisas_komisyonlari_urls.txt", "a+") as f:
    for i in range(len(komisyonlar_df)):
        f.write(f"{komisyonlar_df.loc[i, 'uye_url']}\n")

In [10]:
# Read urls from txt file
with open("urls/ihtisas_komisyonlari_urls.txt", "r") as f:
    urls = [url.strip() for url in f.readlines()]

In [11]:
def get_uyeler(url):
    
    driver.get(url)
    
    uyeler = pd.DataFrame(columns=["adi_ve_soyadi", "unvani", "sehir",
                                   "iletisim_bilgileri", "parti_amblem_url", "ozgecmis_url"])
    
    rows = driver.find_element(By.XPATH, """/html/body/div[2]/div/div/div[1]/div/div/div""").find_elements(By.CLASS_NAME, "col-md-9")
    
    for i, row in enumerate(rows):
        sections = row.find_elements(By.CLASS_NAME, "row")
        
        adi_ve_soyadi = sections[0].text.split("\n")[0]
        unvani = sections[0].text.split("\n")[1]
        sehir = sections[0].text.split("\n")[2]
        iletisim_bilgileri = sections[1].text
        parti_amblem_url = sections[2].find_element(By.TAG_NAME, "img").get_attribute("src")
        ozgecmis_url = sections[3].find_element(By.TAG_NAME, "a").get_attribute("href")
        
        uye = pd.Series([adi_ve_soyadi, unvani, sehir,
                         iletisim_bilgileri, parti_amblem_url, ozgecmis_url],
                        index=["adi_ve_soyadi", "unvani", "sehir",
                               "iletisim_bilgileri", "parti_amblem_url", "ozgecmis_url"])
        
        uyeler = uyeler.append(uye, ignore_index=True)
        
    uyeler = uyeler.reset_index(drop=True)
        
    return uyeler

In [12]:
ihtisas_komisyonlari_uyeler = pd.DataFrame(columns=["adi_ve_soyadi", "unvani", "sehir",
                                   "iletisim_bilgileri", "parti_amblem_url", "ozgecmis_url", "komisyon_id"])

for i in range(len(komisyonlar_df)):
    if i % 5 == 0:
        print(f"{i}/{len(komisyonlar_df)} | {i/len(komisyonlar_df)*100:.2f}%")
    uyeler = get_uyeler(komisyonlar_df.loc[i, "uye_url"])
    uyeler["komisyon_id"] = komisyonlar_df.loc[i, "komisyon_id"]
    ihtisas_komisyonlari_uyeler = ihtisas_komisyonlari_uyeler.append(uyeler, ignore_index=True)
print(f"{len(komisyonlar_df)}/{len(komisyonlar_df)} | {len(komisyonlar_df)/len(komisyonlar_df)*100:.2f}%")

0/19 | 0.00%
5/19 | 26.32%
10/19 | 52.63%
15/19 | 78.95%
19/19 | 100.00%


In [13]:
def parti_amblem_converter(amblem_url):
    return_val = ""
    
    if "YPK29" in amblem_url:
        return_val = "ADALET VE KALKINMA PARTİSİ"
    elif "YPK1" in amblem_url:
        return_val = "CUMHURİYET HALK PARTİSİ"
    elif "YPK74" in amblem_url:
        return_val = "HALKLARIN DEMOKRATİK PARTİSİ"
    elif "YPK2" in amblem_url:
        return_val = "MİLLİYETÇİ HAREKET PARTİSİ"
    elif "YPK81" in amblem_url:
        return_val = "İYİ PARTİ"
    elif "YPK13" in amblem_url:
        return_val = "TÜRKİYE İŞÇİ PARTİSİ"
    elif "YPK75" in amblem_url:
        return_val = "DEMOKRATİK BÖLGELER PARTİSİ"
        
    return return_val

In [14]:
ihtisas_komisyonlari_uyeler["partisi"] = ihtisas_komisyonlari_uyeler["parti_amblem_url"].apply(parti_amblem_converter)
ihtisas_komisyonlari_uyeler["secim_cevresi"] = ihtisas_komisyonlari_uyeler["sehir"].apply(lambda x: x.split()[0].replace("I", "ı").capitalize())

In [15]:
ihtisas_komisyonlari_uyeler.to_csv("ihtisas_komisyonlari/ihtisas_komisyonlari_uyeler.csv", index=False)